In [0]:
from pyspark.sql.types import *

In [0]:
schema_defined = StructType([StructField("File",StringType(),True)
                             ,StructField("Shop",StringType(),True)
                             ,StructField("Sales_count",IntegerType(),True)])
                            


In [0]:
dbutils.fs.rm("/Filestore/stream/stream_checkpoint1/",True)
dbutils.fs.rm("/Filestore/stream/stream_read1/",True)
dbutils.fs.rm("/Filestore/stream/stream_write1/",True)

dbutils.fs.mkdirs("/Filestore/stream/stream_checkpoint1/")
dbutils.fs.mkdirs("/Filestore/stream/stream_read1/")
dbutils.fs.mkdirs("/Filestore/stream/stream_write1/")

Out[13]: True

In [0]:
display(df)

File,Shop,Sales_count
File1,Amazon,100
File1,Flipkart,50
File1,Myntra,20
File1,Snapdeal,10
File1,Alibaba,null
File2,Amazon,200
File2,Flipkart,50
File2,Myntra,20
File2,Snapdeal,20
File2,Alibaba,null


In [0]:
df = spark.readStream.format("csv").schema(schema_defined).option("header",True).option("sep",";").load("/FileStore/Filestore/stream/stream_read1")

df1 = df.groupBy("Shop").sum("Sales_count")
display(df1)

Shop,sum(Sales_count)
Snapdeal,195
Myntra,67
Flipkart,166
Alibaba,234
Amazon,600


In [0]:
df4 = df.writeStream.format("parquet").outputMode("append").option("path","/FileStore/Filestore/stream/stream_write1/").option("checkpointLocation","/FileStore/Filestore/stream/stream_checkpoint1/").start().awaitTermination()



In [0]:
display(spark.read.format("parquet").load("/FileStore/Filestore/stream/stream_write1/*.parquet"))